# Laboratoire 3 _ CEG 4536


# Tache 3 - Implémentation pratique

3. Utilisation de la mémoire partagée

In [1]:
!nvidia-smi

Mon Nov 25 02:31:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
%%writefile Tache3.cu
#include <stdio.h>
#include <stdlib.h>

#define N 1024  // Taille de la matrice
#define TILE_SIZE 32  // Taille des tuiles pour la mémoire partagée

// Noyau CUDA pour la multiplication matricielle avec mémoire partagée
__global__ void matrixMultiplyShared(float *a, float *b, float *c, int n) {
    __shared__ float tileA[TILE_SIZE][TILE_SIZE];
    __shared__ float tileB[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Calcul de la multiplication matricielle
    float sum = 0.0f;
    for (int t = 0; t < (n + TILE_SIZE - 1) / TILE_SIZE; t++) {
      // Chargement des tuiles de A et B dans la mémoire partagée
      if (row < n && t * TILE_SIZE + threadIdx.x < n) {
            tileA[threadIdx.y][threadIdx.x] = a[row * n + t * TILE_SIZE + threadIdx.x];
        } else {
            tileA[threadIdx.y][threadIdx.x] = 0.0f;
        }

        if (col < n && t * TILE_SIZE + threadIdx.y < n) {
            tileB[threadIdx.y][threadIdx.x] = b[(t * TILE_SIZE + threadIdx.y) * n + col];
        } else {
            tileB[threadIdx.y][threadIdx.x] = 0.0f;
        }

        __syncthreads();

        // Calculer la multiplication de la tuile
        for (int k = 0; k < TILE_SIZE; ++k) {
            sum += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];
        }

        __syncthreads();
    }

    // Ecrire le résultat dans C
    if (row < n && col < n) {
        c[row * n + col] = sum;
    }
}

int main(){
    int size = N * N * sizeof(float);
    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    // Allocation de mémoire sur l'hôte
    h_A = (float*)malloc(size);
    h_B = (float*)malloc(size);
    h_C = (float*)malloc(size);

    // Initialisation des matrices A et B
    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() % 100 / 10.0f;  // Valeurs aléatoires entre 0 et 10
        h_B[i] = rand() % 100 / 10.0f;
    }

    // Allocation de mémoire sur le GPU
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copie des matrices de l'hôte au GPU
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Lancement du noyau CUDA
    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    dim3 numBlocks((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);
    matrixMultiplyShared<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Copie des résultats du GPU vers l'hôte
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Affichage d'une partie des résultats
    printf("Matrice C (extrait) :\n");
    for (int i = 0; i < 5; i++) {
        for (int j = 0; j < 5; j++) {
            printf("%0.2f ", h_C[i * N + j]);
        }
        printf("\n");
    }

    // Libération de la mémoire
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;

}

Overwriting Tache3.cu


In [8]:
!nvcc Tache3.cu -o Tache3

In [9]:
!./Tache3

Matrice C (extrait) :
25344.94 25270.46 25240.65 25736.77 24979.00 
25968.35 25222.31 25647.50 25935.41 24874.16 
25986.41 26152.93 25719.11 26166.12 25138.52 
25745.83 25380.14 25408.12 26184.89 25720.78 
24591.71 24641.83 24440.13 24807.35 23743.05 


In [10]:
!nvprof ./Tache3

==2325== NVPROF is profiling process 2325, command: ./Tache3
Matrice C (extrait) :
25344.94 25270.46 25240.65 25736.77 24979.00 
25968.35 25222.31 25647.50 25935.41 24874.16 
25986.41 26152.93 25719.11 26166.12 25138.52 
25745.83 25380.14 25408.12 26184.89 25720.78 
24591.71 24641.83 24440.13 24807.35 23743.05 
==2325== Profiling application: ./Tache3
==2325== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   58.54%  5.3674ms         1  5.3674ms  5.3674ms  5.3674ms  matrixMultiplyShared(float*, float*, float*, int)
                   22.65%  2.0765ms         1  2.0765ms  2.0765ms  2.0765ms  [CUDA memcpy DtoH]
                   18.81%  1.7247ms         2  862.37us  859.33us  865.41us  [CUDA memcpy HtoD]
      API calls:   94.37%  203.25ms         3  67.751ms  68.098us  203.11ms  cudaMalloc
                    5.16%  11.106ms         3  3.7021ms  1.0783ms  8.8884ms  cudaMemcpy
                    0.25%  548.68us       

# Resultat et explication

Les résultats du profilage montrent que l'optimisation de la multiplication matricielle à l'aide de la mémoire partagée dans le noyau matrixMultiplyShared a considérablement amélioré les performances. Le noyau CUDA représente 58.54% du temps total d'exécution GPU, avec un temps d'exécution de 5.3674 ms, confirmant une accélération significative par rapport aux implémentations précédentes utilisant uniquement la mémoire globale.
Les transferts de données entre l'hôte et le périphérique (HtoD) ont consommé 18.81% du temps total, soit 1.7247 ms répartis sur deux appels, tandis que les transferts du périphérique vers l'hôte (DtoH) ont occupé 22.65%, soit 2.0765 ms pour un seul appel.
Ces transferts restent un goulot d'étranglement malgré l'amélioration des calculs.
Du côté des appels d'API CUDA, cudaMalloc domine avec 94.37% du temps total API (203.25 ms répartis sur trois appels), ce qui reflète le coût élevé des allocations de mémoire sur le GPU. Les appels cudaMemcpy représentent 5.16% (11.106 ms pour trois appels), montrant que la gestion des transferts pourrait encore être optimisée.
Les autres appels, comme cudaFree et cudaLaunchKernel, ne représentent qu'une fraction négligeable du temps total, témoignant d'une gestion efficace en dehors des principales opérations.

En conclusion, l'utilisation de la mémoire partagée dans cette implémentation a réduit de manière significative le temps d'exécution du calcul principal, mais les transferts mémoire et l'allocation GPU demeurent des axes d'amélioration potentiels pour optimiser davantage les performances globales.

# -------------------------------------------------


# Tache 4 - Profilage et évaluation

4. Optimiser les schémas d'accès à la mémoire partagée

In [15]:
%%writefile Tache4.cu
#include <stdio.h>
#include <stdlib.h>

#define BLOCK_SIZE 32
#define PADDING 1 // pour eviter les blocs trop petits

__global__ void matrixMultiplyShared(float* A, float* B, float* C, int N) {
    __shared__ float tileA[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ float tileB[BLOCK_SIZE][BLOCK_SIZE];

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;

    for (int m = 0; m < (N + BLOCK_SIZE - 1) / BLOCK_SIZE; ++m) {
        if (row < N && (m * BLOCK_SIZE + threadIdx.x) < N)
            tileA[threadIdx.y][threadIdx.x] = A[row * N + (m * BLOCK_SIZE + threadIdx.x)];
        else
            tileA[threadIdx.y][threadIdx.x] = 0.0f;

        if (col < N && (m * BLOCK_SIZE + threadIdx.y) < N)
            tileB[threadIdx.y][threadIdx.x] = B[(m * BLOCK_SIZE + threadIdx.y) * N + col];
        else
            tileB[threadIdx.y][threadIdx.x] = 0.0f;

        __syncthreads();

        for (int k = 0; k < BLOCK_SIZE; ++k) {
            sum += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < N && col < N)
        C[row * N + col] = sum;
}

int main() {
    int N = 512;  // Taille de la matrice, modifiable selon les besoins
    int size = N * N * sizeof(float);
    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    // Allocation de mémoire sur l'hôte
    h_A = (float*)malloc(size);
    h_B = (float*)malloc(size);
    h_C = (float*)malloc(size);

    // Initialisation des matrices A et B aléatoires
    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() % 100 / 10.0;
        h_B[i] = rand() % 100 / 10.0;
    }

    // Allocation de mémoire sur le GPU
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copie des matrices de l'hôte au GPU
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) { //pour s'assurer que cudaMempcy copie bien les données entre l'hote et le gpu
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
      }

    // Lancement du noyau CUDA
    dim3 threadsPerBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 numBlocks((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);
    matrixMultiplyShared<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Copie des résultats du GPU vers l'hôte
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Affichage d'une partie des résultats
    printf("Matrice C (extrait) :\n");
    for (int i = 0; i < 5; i++) {
        for (int j = 0; j < 5; j++) {
            printf("%0.2f ", h_C[i * N + j]);
        }
        printf("\n");
    }

    // Libération de la mémoire
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Overwriting Tache4.cu


In [16]:
!nvcc Tache4.cu -o Tache4

In [17]:
!./Tache4

Matrice C (extrait) :
12815.51 12810.97 13229.42 12867.26 13151.88 
12641.00 12111.48 12917.40 12138.28 13090.13 
12931.40 12732.82 13043.32 12652.42 13247.57 
12449.74 12155.74 12686.56 12167.19 12716.53 
13361.70 13210.03 13280.66 12809.75 13371.86 


In [18]:
!nvprof ./Tache4

==7913== NVPROF is profiling process 7913, command: ./Tache4
Matrice C (extrait) :
12815.51 12810.97 13229.42 12867.26 13151.88 
12641.00 12111.48 12917.40 12138.28 13090.13 
12931.40 12732.82 13043.32 12652.42 13247.57 
12449.74 12155.74 12686.56 12167.19 12716.53 
13361.70 13210.03 13280.66 12809.75 13371.86 
==7913== Profiling application: ./Tache4
==7913== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   72.92%  716.22us         1  716.22us  716.22us  716.22us  matrixMultiplyShared(float*, float*, float*, int)
                   17.93%  176.13us         2  88.064us  87.584us  88.544us  [CUDA memcpy HtoD]
                    9.14%  89.792us         1  89.792us  89.792us  89.792us  [CUDA memcpy DtoH]
      API calls:   98.51%  189.58ms         3  63.193ms  3.5900us  189.50ms  cudaMalloc
                    1.14%  2.1858ms         3  728.59us  259.41us  1.6105ms  cudaMemcpy
                    0.12%  237.05us       

# Resultats et explications


Les résultats du profilage pour l'exécution de la tâche matrixMultiplyShared montrent des performances optimisées. Le noyau CUDA représente 72.92% du temps total d'exécution GPU, avec un temps d'exécution de 716.22 µs, ce qui illustre une grande efficacité dans l'utilisation de la mémoire partagée. Les transferts mémoire entre l'hôte et le périphérique (HtoD) ont occupé 17.93% du temps total, soit 176.13 µs répartis sur deux appels, tandis que les transferts du périphérique vers l'hôte (DtoH) représentent 9.14%, avec un temps de 89.792 µs pour un appel unique. Ces transferts, bien que significatifs, montrent une amélioration notable par rapport aux implémentations précédentes.

Du côté des appels d'API CUDA, cudaMalloc domine toujours, représentant 98.51% du temps total API (189.58 ms répartis sur trois appels), ce qui reflète l'impact des allocations mémoire sur les performances globales. Les appels cudaMemcpy représentent 1.14% du temps total API (2.1858 ms pour trois appels), tandis que les autres appels, comme cudaFree et cudaLaunchKernel, occupent une part négligeable, avec respectivement 0.12% et 0.12% du temps total API. Cela témoigne d'une gestion efficace des ressources hors des opérations principales.

En conclusion, l'utilisation de la mémoire partagée dans cette tâche a permis de réduire significativement le temps d'exécution du calcul principal à 716.22 µs, mais les transferts mémoire et les allocations GPU restent des points à optimiser davantage. Ces résultats montrent que la stratégie d'optimisation actuelle est efficace pour améliorer les performances globales.